In [1]:
import sys
import pandas as pd
import numpy as np
import re
import pywikibot
import requests
from bs4 import BeautifulSoup
import datetime


In [ ]:
catalog_df = pd.read_csv("public_domain_dump-master/pseudocatalogue.csv")
i = 6
i

In [ ]:
ONLY_WORDS_AND_DIGITS_REGEX = re.compile(r'[\w\dא-ת]+')

def get_only_words_and_digits(text: str):
    return ONLY_WORDS_AND_DIGITS_REGEX.findall(text)

def invert_words(words: list):
    return [w[::-1] for w in words]

def remove_last_line_from_string(s):
    s = s.split("את הטקסט לעיל הפיקו מתנדבי פרויקט בן־יהודה באינטרנט.  הוא זמין תמיד בכתובת הבאה")
    return s[0]


def remove_author_title(s, author, title):
    s = s.replace(title, "", 1)
    return s.replace(author, "")

def normalize_text(s):
    s = re.sub(r'[\u0591-\u05BD\u05BF-\u05C2\u05C4-\u05C7]', '', s)
    return s.strip()

def parse_place(place):
    place = place.replace("\n", "$")
    place = place.split("$$$$$$$$$$$$$$$$")[1]
    return place.split("$$$$$$$$$")[0]

def parse_date(date):
    date = date.replace("\n", "$")
    date = date.split("$$$$$$$$$$$$$$$$")[1]
    date = date.split("$$$$$$$$$")[0]
    date = date.split("Gregorian")[0]

    if len(date) == 4:
        return datetime.datetime.strptime(date, '%Y')
    return  datetime.datetime.strptime(date, '%d %B %Y')


In [ ]:
i = 0
content = []
for index, row in catalog_df.iterrows():
    print(index)
    path = row["path"]
    title = normalize_text(row["title"])
    author = row["authors"]
    path = "public_domain_dump-master/txt_stripped" + path + ".txt"
    with open(path, "r", encoding="utf-8") as f:
        text = f.read()
        text = remove_last_line_from_string(text)
        text = remove_author_title(text, author, title)
        text = get_only_words_and_digits(text)
        content.append(text)
    i += 1
catalog_df["text"] = content

In [ ]:
def parse_writer(writer, id):
    site = pywikibot.Site("he", "wikipedia")
    page = pywikibot.Page(site, writer)
    try:
        item = pywikibot.ItemPage.fromPage(page)
        page_id = item.id
        URL = f"https://www.wikidata.org/wiki/{page_id}"
    except:

        URL = f"https://he.wikipedia.org/?curid={page.pageid}"
        response = requests.get(URL)
        soup = BeautifulSoup(response.text, 'html')
        URL = soup.find('a', {'title' : 'קישור לפריט המשויך במאגר הנתונים [g]'})['href']


    response = requests.get(URL)
    print(response, writer)

    soup = BeautifulSoup(response.text, 'html')


    try:
        sex = soup.find('a', {'title' : 'Q6581097'}).string
    except:
        sex = "female"


    place_of_birth = soup.find("div", {"id": "P19"}).text
    try:
        place_of_birth = parse_place(place_of_birth)
    except:
        print("p birth - ", sys.exc_info())

    place_of_death = soup.find("div", {"id": "P20"}).text
    try:
        place_of_death = parse_place(place_of_death)
    except:
        print("p death - ", sys.exc_info())


    date_of_birth = soup.find("div", {"id": "P569"}).text
    try:
        date_of_birth = parse_date(date_of_birth)
    except:
        print("d birth - ", sys.exc_info())

    date_of_death = soup.find("div", {"id": "P570"}).text
    try:
        date_of_death = parse_date(date_of_death)
    except:
        print("d death - ", sys.exc_info())

    return [id, writer, sex, place_of_birth, place_of_death, date_of_birth, date_of_death]

In [ ]:
writers = catalog_df["authors"].unique()
authors_df = pd.DataFrame(columns=["id", "name", "sex", "p_birth", "p_death", "d_birth", "d_death"], )
for i, writer in enumerate(writers):
    try:
        authors_df.loc[i] = parse_writer(writer, i)
    except Exception:
        authors_df.loc[i] = [i, writer, None, None, None, None, None]
        print(sys.exc_info())
        print("couldn't complete for", writer, i)
print(authors_df)
authors_df.to_csv("authors", index=False)

In [2]:
def get_year_category(year):
    if year <= 500:
        return 0
    if 500 < year <= 1400:
        return 1
    if 1400 < year <= 1800:
        return 2
    if 1800 < year <= 1900:
        return 3
    return 4

In [3]:
from dateutil import parser

def get_year(year_str):
    try:
        if year_str and year_str is not np.nan:
            if '0000' in year_str:
                return 0
            return parser.parse(year_str).year
    except:
        return None

In [15]:

def parse_authors(df, cities):
    data = []
    for index, row in df.iterrows():
        name = row["name"]
        id = row["id"]
        male = 1 if row["sex"] == "male" else 0
        birth_year = get_year(row["d_birth"])
        death_year = get_year(row["d_death"])
        if birth_year is None or death_year is None:
            print(row["name"])
            continue
        birth = get_year_category(birth_year)
        death = get_year_category(death_year)
        hebrew_speaker = 1 if "FALSE" != row["Hebrew Speaker"] else 0
        p_birth_israel = 1 if row["p_birth"] in cities else 0
        p_death_israel = 1 if row["p_death"] in cities else 0

        data.append([name, id, male, birth, death, hebrew_speaker, p_birth_israel, p_death_israel])
    return data




poetry_df = pd.read_csv('Data/poetry.csv')
poetry_df = poetry_df[poetry_df['genre'] == "שירה"]
poetry_names = set(poetry_df.authors)
authors_df = pd.read_csv('authors.csv')

authors_df = authors_df[authors_df['name'].isin(poetry_names)]


israeli_cities = ["Tel Aviv", "Jaffa", "Jerusalem", "Haifa", "Nahalal", "Acre", "Ramat Gan", "Givat Hashlosha",
                  "Rishon LeZion", "Petah Tikva", "Zikhron Ya'akov", "Israel", "Mandatory Palestine"]

parsed_authors_data = parse_authors(authors_df, israeli_cities)
parsed_authors_df = pd.DataFrame(parsed_authors_data, columns=["name", "id", "male", "birth", "death", "hebrew_speaker", "p_birth_israel", "p_death_israel"])

אלמוני/ת
יוסף בן שמואל צרפתי
אינוקנטי אננסקי
חיא
יצחק בעל “עזרת נשים”


In [17]:
parsed_authors_df.to_csv("authors_parsed.csv", index=False)

In [24]:
poetry_df = poetry_df[["authors", "content"]]
poetry_df.drop(columns=["authors"])
merged_df = parsed_authors_df.merge(poetry_df, left_on="name", right_on="authors")

,name,id,male,birth,death,hebrew_speaker,p_birth_israel,p_death_israel,authors,content
0,יהודה ליב גורדון,2,1,3,3,1,0,0,יהודה ליב גורדון,\t \t \t לידידי הרב החכם רב-הפעלים מ...
1,יהודה ליב גורדון,2,1,3,3,1,0,0,יהודה ליב גורדון,\t \t \t נפישי גמלי סבי דטעיני משכי דהוג...
2,יהודה ליב גורדון,2,1,3,3,1,0,0,יהודה ליב גורדון,"\t \t \t [על לוח תמונתו] לאיס""י הבבלי,..."
3,יהודה ליב גורדון,2,1,3,3,1,0,0,יהודה ליב גורדון,\t \t \t על אדמת בנימין בעיר בגבע שם...
4,יהודה ליב גורדון,2,1,3,3,1,0,0,יהודה ליב גורדון,\t \t \t אחרי גבר חכם יודע רדף ברחוב...
...,...,...,...,...,...,...,...,...,...,...
6983,קורנל אוייסקי,445,1,3,3,1,0,0,קורנל אוייסקי,"\t \t \t מרשפי השמש נשזפתי, השחרתי… וי..."
6984,קורנל אוייסקי,445,1,3,3,1,0,0,קורנל אוייסקי,\t \t \t הכשדים – לדמים כנמרים ישאפו – ...
6985,קורנל אוייסקי,445,1,3,3,1,0,0,קורנל אוייסקי,"\t \t \t לו עדי, נאם בלעם, מואבים ישמעו..."
6986,שרה שפירא,450,0,3,4,1,0,0,שרה שפירא,"\t \t \t 1 חושו הצילו, מלטו עניה! ..."


In [25]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(merged_df, test_size=0.3)
validation, test = train_test_split(test, train_size=0.5)


,name,id,male,birth,death,hebrew_speaker,p_birth_israel,p_death_israel,authors,content
2154,שמואל הנגיד,32,1,1,1,1,0,0,שמואל הנגיד,\t \t \t אם יעשיר איש סכל הוחל מעט י...
5554,יעקב שטיינברג,261,1,3,4,1,0,1,יעקב שטיינברג,\t \t \t א בקרב חצות ליל קר ומעט שאון ...
5048,יצחק אבן כלפון,113,1,1,1,1,0,0,יצחק אבן כלפון,"\t \t \t שפטני אלהי הצבאות, ושמני במ..."
1401,יהודה הלוי,26,1,1,1,1,0,1,יהודה הלוי,\t \t \t יונת רחוקים נדדה יערה / ר' ...
5303,דוד פוגל,206,1,3,4,1,0,0,דוד פוגל,"\t \t \t יומנו החור לאינו למכונות, עתה..."
...,...,...,...,...,...,...,...,...,...,...
6972,אברהם בן־יצחק,435,1,3,4,1,0,1,אברהם בן־יצחק,\t \t \t שגעון עינים ולא תנודת רגל פנ...
832,משה אבן עזרא,11,1,1,1,1,0,0,משה אבן עזרא,\t \t \t מאס תבל ואל־יום הפקדה / בכל־עת...
4839,יעקב לרנר,79,1,3,4,1,0,0,יעקב לרנר,\t \t \t (לזכר אמי) היא פתחה הדלת כי...
2726,שלמה אבן גבירול,40,1,1,1,1,0,0,שלמה אבן גבירול,"\t \t \t לחולת אהבה נפשי שלולה, / בעולת..."


In [27]:
train.to_csv("train.csv")
test.to_csv("test.csv")
validation.to_csv("validation.csv")


In [34]:
print(test[test["p_death_israel"] == 0].shape[0])
print(test.shape[0])

730
1049
